# Project Development

## Learning Objectives
By the end of this lesson, you will be able to:
- Structure Python projects with proper organization
- Write and run unit tests using pytest
- Handle logging and debugging effectively
- Create command-line interfaces
- Package and distribute Python applications

## Core Concepts
- **Project Structure**: Organizing code, tests, and documentation
- **Testing**: Automated testing with pytest
- **Logging**: Recording application events and errors
- **CLI**: Command-line interface development
- **Packaging**: Creating distributable Python packages

# 1. Project Structure and Organization

In [ ]:
import os
from pathlib import Path

# Create a sample project structure
def create_project_structure(project_name):
    """Create a standard Python project structure"""
    
    project_structure = {
        f"{project_name}/": {},
        f"{project_name}/src/": {},
        f"{project_name}/src/{project_name}/": {},
        f"{project_name}/tests/": {},
        f"{project_name}/docs/": {},
        f"{project_name}/scripts/": {},
    }
    
    files_to_create = {
        f"{project_name}/README.md": f"# {project_name}\n\nProject description goes here.",
        f"{project_name}/requirements.txt": "# Add your dependencies here\n",
        f"{project_name}/setup.py": f"""
from setuptools import setup, find_packages

setup(
    name="{project_name}",
    version="0.1.0",
    packages=find_packages(where="src"),
    package_dir={{"": "src"}},
    install_requires=[],
)
""",
        f"{project_name}/src/{project_name}/__init__.py": f'"""\\n{project_name} package\\n"""\\n\\n__version__ = "0.1.0"',
        f"{project_name}/src/{project_name}/main.py": '''
"""
Main module for the application
"""

def main():
    """Main entry point"""
    print("Hello from main!")

if __name__ == "__main__":
    main()
''',
        f"{project_name}/tests/__init__.py": "",
        f"{project_name}/tests/test_main.py": f'''
"""
Tests for {project_name}.main
"""

import pytest
from {project_name}.main import main

def test_main():
    """Test main function"""
    # This would normally test actual functionality
    assert main() is None
''',
        f"{project_name}/.gitignore": '''
__pycache__/
*.pyc
*.pyo
*.pyd
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg
''',
    }
    
    # Create directories
    for directory in project_structure:
        Path(directory).mkdir(parents=True, exist_ok=True)
    
    # Create files
    for filepath, content in files_to_create.items():
        with open(filepath, 'w') as f:
            f.write(content.strip())
    
    return f"Created project structure for {project_name}"

# Create sample project
result = create_project_structure("myapp")
print(result)

# Display project structure
def show_tree(directory, prefix="", max_depth=3, current_depth=0):
    """Display directory tree structure"""
    if current_depth >= max_depth:
        return
    
    path = Path(directory)
    if not path.exists():
        return
    
    items = sorted(path.iterdir())
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        if item.is_dir() and current_depth < max_depth - 1:
            extension = "    " if is_last else "│   "
            show_tree(item, prefix + extension, max_depth, current_depth + 1)

print("\\nProject structure:")
show_tree("myapp")

# Configuration management
config_content = '''
"""
Configuration management
"""

import os
import json
from pathlib import Path

class Config:
    """Application configuration"""
    
    def __init__(self, config_file=None):
        self.config_file = config_file or "config.json"
        self.settings = self.load_config()
    
    def load_config(self):
        """Load configuration from file"""
        default_config = {
            "app_name": "MyApp",
            "debug": False,
            "database_url": "sqlite:///app.db",
            "log_level": "INFO"
        }
        
        if Path(self.config_file).exists():
            try:
                with open(self.config_file, 'r') as f:
                    file_config = json.load(f)
                default_config.update(file_config)
            except (json.JSONDecodeError, IOError):
                print(f"Warning: Could not load {self.config_file}")
        
        return default_config
    
    def get(self, key, default=None):
        """Get configuration value"""
        return self.settings.get(key, default)
    
    def save_config(self):
        """Save current configuration to file"""
        with open(self.config_file, 'w') as f:
            json.dump(self.settings, f, indent=2)

# Example usage
config = Config()
print(f"App name: {config.get('app_name')}")
print(f"Debug mode: {config.get('debug')}")
'''

with open("myapp/src/myapp/config.py", "w") as f:
    f.write(config_content)

print("\\nAdded configuration management")

# 2. Testing and Debugging

In [ ]:
# Testing with unittest (built-in)
import unittest
import logging

# Sample module to test
class Calculator:
    """Simple calculator for testing"""
    
    def add(self, a, b):
        return a + b
    
    def divide(self, a, b):
        if b == 0:
            raise ValueError("Cannot divide by zero")
        return a / b
    
    def factorial(self, n):
        if n < 0:
            raise ValueError("Factorial not defined for negative numbers")
        if n <= 1:
            return 1
        return n * self.factorial(n - 1)

# Test cases
class TestCalculator(unittest.TestCase):
    """Test cases for Calculator class"""
    
    def setUp(self):
        """Set up test fixtures before each test method"""
        self.calc = Calculator()
    
    def test_add(self):
        """Test addition"""
        self.assertEqual(self.calc.add(2, 3), 5)
        self.assertEqual(self.calc.add(-1, 1), 0)
    
    def test_divide(self):
        """Test division"""
        self.assertEqual(self.calc.divide(10, 2), 5)
        self.assertAlmostEqual(self.calc.divide(1, 3), 0.333333, places=5)
    
    def test_divide_by_zero(self):
        """Test division by zero raises exception"""
        with self.assertRaises(ValueError):
            self.calc.divide(10, 0)
    
    def test_factorial(self):
        """Test factorial calculation"""
        self.assertEqual(self.calc.factorial(0), 1)
        self.assertEqual(self.calc.factorial(5), 120)
        
        with self.assertRaises(ValueError):
            self.calc.factorial(-1)

# Run tests (simulation)
if __name__ == "__main__":
    # Create test suite
    suite = unittest.TestLoader().loadTestsFromTestCase(TestCalculator)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    
    print(f"\\nTests run: {result.testsRun}")
    print(f"Failures: {len(result.failures)}")
    print(f"Errors: {len(result.errors)}")

# Logging setup
def setup_logging(log_level=logging.INFO):
    """Configure logging for the application"""
    
    # Create logs directory
    Path("logs").mkdir(exist_ok=True)
    
    # Configure logging
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('logs/app.log'),
            logging.StreamHandler()
        ]
    )
    
    return logging.getLogger(__name__)

# Example with logging
logger = setup_logging()

class DataProcessor:
    """Example class with logging"""
    
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def process_data(self, data):
        """Process data with logging"""
        self.logger.info(f"Processing {len(data)} items")
        
        try:
            processed = []
            for item in data:
                if isinstance(item, (int, float)):
                    result = item * 2
                    processed.append(result)
                    self.logger.debug(f"Processed {item} -> {result}")
                else:
                    self.logger.warning(f"Skipped invalid item: {item}")
            
            self.logger.info(f"Successfully processed {len(processed)} items")
            return processed
            
        except Exception as e:
            self.logger.error(f"Error processing data: {e}")
            raise

# Test data processing with logging
processor = DataProcessor()
test_data = [1, 2, "invalid", 3.5, None, 4]
result = processor.process_data(test_data)
print(f"Processed result: {result}")

# Debugging techniques
def debug_example():
    """Example function for debugging"""
    
    # Using assert for debugging
    def validate_positive(value):
        assert value > 0, f"Value must be positive, got {value}"
        return value
    
    # Using pdb for debugging (commented out for notebook)
    # import pdb; pdb.set_trace()
    
    # Debugging with print statements
    numbers = [1, 2, 3, 4, 5]
    print(f"DEBUG: Initial numbers: {numbers}")
    
    squared = [x**2 for x in numbers]
    print(f"DEBUG: Squared numbers: {squared}")
    
    # Using logging for debugging
    logger.debug("This is a debug message")
    logger.info("Processing complete")
    
    return squared

debug_result = debug_example()
print(f"Debug example result: {debug_result}")

# 3. Command-Line Interfaces

In [ ]:
import argparse
import sys
from pathlib import Path

# Simple CLI with argparse
def create_cli_parser():
    """Create command-line argument parser"""
    
    parser = argparse.ArgumentParser(
        description="File Management Tool",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Examples:
  python cli.py count --directory /path/to/files
  python cli.py search --pattern "*.py" --directory src/
        """
    )
    
    # Subcommands
    subparsers = parser.add_subparsers(dest='command', help='Available commands')
    
    # Count command
    count_parser = subparsers.add_parser('count', help='Count files in directory')
    count_parser.add_argument('--directory', '-d', default='.', 
                             help='Directory to count files in')
    count_parser.add_argument('--recursive', '-r', action='store_true',
                             help='Count files recursively')
    
    # Search command
    search_parser = subparsers.add_parser('search', help='Search for files')
    search_parser.add_argument('--pattern', '-p', required=True,
                              help='File pattern to search for')
    search_parser.add_argument('--directory', '-d', default='.',
                              help='Directory to search in')
    
    # Info command
    info_parser = subparsers.add_parser('info', help='Show file information')
    info_parser.add_argument('filename', help='File to show info for')
    
    return parser

# CLI functions
def count_files(directory, recursive=False):
    """Count files in directory"""
    path = Path(directory)
    
    if not path.exists():
        return f"Directory {directory} does not exist"
    
    if recursive:
        files = list(path.rglob('*'))
        file_count = len([f for f in files if f.is_file()])
    else:
        files = list(path.iterdir())
        file_count = len([f for f in files if f.is_file()])
    
    return f"Found {file_count} files in {directory}" + (" (recursive)" if recursive else "")

def search_files(pattern, directory):
    """Search for files matching pattern"""
    path = Path(directory)
    
    if not path.exists():
        return f"Directory {directory} does not exist"
    
    matches = list(path.glob(pattern))
    
    if matches:
        result = f"Found {len(matches)} files matching '{pattern}':\\n"
        for match in matches:
            result += f"  {match}\\n"
        return result
    else:
        return f"No files found matching '{pattern}'"

def show_file_info(filename):
    """Show information about a file"""
    path = Path(filename)
    
    if not path.exists():
        return f"File {filename} does not exist"
    
    stat = path.stat()
    info = f"""
File: {path.name}
Path: {path.absolute()}
Size: {stat.st_size} bytes
Type: {'Directory' if path.is_dir() else 'File'}
"""
    return info

# Simulate CLI execution
def simulate_cli(args_list):
    """Simulate CLI execution with given arguments"""
    parser = create_cli_parser()
    
    # Parse arguments
    args = parser.parse_args(args_list)
    
    if args.command == 'count':
        return count_files(args.directory, args.recursive)
    elif args.command == 'search':
        return search_files(args.pattern, args.directory)
    elif args.command == 'info':
        return show_file_info(args.filename)
    else:
        return "No command specified"

# Test CLI commands
print("CLI Examples:")
print("1. Count files:")
print(simulate_cli(['count', '--directory', '.']))

print("\\n2. Search for Python files:")
print(simulate_cli(['search', '--pattern', '*.py', '--directory', '.']))

print("\\n3. Show file info:")
# Create a test file first
test_file = "test_cli.txt"
with open(test_file, "w") as f:
    f.write("Test file for CLI demo")

print(simulate_cli(['info', test_file]))

# Advanced CLI with configuration
class CLIApp:
    """More advanced CLI application"""
    
    def __init__(self):
        self.config = {
            'verbose': False,
            'output_format': 'text'
        }
    
    def setup_parser(self):
        """Setup argument parser with global options"""
        parser = argparse.ArgumentParser(description="Advanced CLI App")
        
        # Global options
        parser.add_argument('--verbose', '-v', action='store_true',
                           help='Enable verbose output')
        parser.add_argument('--format', choices=['text', 'json'],
                           default='text', help='Output format')
        
        # Subcommands
        subparsers = parser.add_subparsers(dest='command')
        
        # Process command
        process_cmd = subparsers.add_parser('process')
        process_cmd.add_argument('input_file', help='Input file to process')
        process_cmd.add_argument('--output', '-o', help='Output file')
        
        return parser
    
    def run(self, args_list):
        """Run the CLI application"""
        parser = self.setup_parser()
        args = parser.parse_args(args_list)
        
        # Update config with parsed args
        self.config['verbose'] = args.verbose
        self.config['output_format'] = args.format
        
        if self.config['verbose']:
            print(f"Running in verbose mode, format: {self.config['output_format']}")
        
        if args.command == 'process':
            return self.process_file(args.input_file, args.output)
        
        return "No valid command"
    
    def process_file(self, input_file, output_file=None):
        """Process input file"""
        if self.config['verbose']:
            print(f"Processing file: {input_file}")
        
        # Simulate file processing
        result = f"Processed {input_file}"
        
        if output_file:
            result += f" -> {output_file}"
        
        return result

# Test advanced CLI
app = CLIApp()
print("\\nAdvanced CLI Example:")
print(app.run(['--verbose', '--format', 'json', 'process', 'input.txt', '--output', 'output.txt']))

# Practice Exercises

In [ ]:
# Exercise 1: Task Management Application
import json
import datetime
from pathlib import Path
import uuid

class TaskManager:
    """Simple task management system"""
    
    def __init__(self, data_file="tasks.json"):
        self.data_file = data_file
        self.tasks = self.load_tasks()
    
    def load_tasks(self):
        """Load tasks from file"""
        if Path(self.data_file).exists():
            try:
                with open(self.data_file, 'r') as f:
                    return json.load(f)
            except (json.JSONDecodeError, IOError):
                return {}
        return {}
    
    def save_tasks(self):
        """Save tasks to file"""
        with open(self.data_file, 'w') as f:
            json.dump(self.tasks, f, indent=2, default=str)
    
    def add_task(self, title, description="", priority="medium"):
        """Add a new task"""
        task_id = str(uuid.uuid4())[:8]
        task = {
            "id": task_id,
            "title": title,
            "description": description,
            "priority": priority,
            "status": "pending",
            "created": datetime.datetime.now(),
            "completed": None
        }
        self.tasks[task_id] = task
        self.save_tasks()
        return task_id
    
    def complete_task(self, task_id):
        """Mark task as completed"""
        if task_id in self.tasks:
            self.tasks[task_id]["status"] = "completed"
            self.tasks[task_id]["completed"] = datetime.datetime.now()
            self.save_tasks()
            return True
        return False
    
    def list_tasks(self, status=None):
        """List tasks, optionally filtered by status"""
        filtered_tasks = []
        for task in self.tasks.values():
            if status is None or task["status"] == status:
                filtered_tasks.append(task)
        
        return sorted(filtered_tasks, key=lambda x: x["created"], reverse=True)
    
    def delete_task(self, task_id):
        """Delete a task"""
        if task_id in self.tasks:
            del self.tasks[task_id]
            self.save_tasks()
            return True
        return False

# Test task manager
tm = TaskManager("demo_tasks.json")

# Add some tasks
task1_id = tm.add_task("Learn Python", "Complete Python tutorial", "high")
task2_id = tm.add_task("Build project", "Create a web application", "medium")
task3_id = tm.add_task("Write tests", "Add unit tests for project", "high")

print("Added tasks:")
for task in tm.list_tasks():
    print(f"- {task['title']} ({task['priority']}) - {task['status']}")

# Complete a task
tm.complete_task(task1_id)

print("\\nAfter completing first task:")
for task in tm.list_tasks():
    status_info = f"{task['status']}"
    if task['status'] == 'completed':
        status_info += f" on {task['completed'].strftime('%Y-%m-%d')}"
    print(f"- {task['title']} - {status_info}")

# Exercise 2: Log Analysis Tool
import re
from collections import Counter, defaultdict

class LogAnalyzer:
    """Analyze log files and extract insights"""
    
    def __init__(self):
        self.log_pattern = re.compile(
            r'(?P<timestamp>\\d{4}-\\d{2}-\\d{2} \\d{2}:\\d{2}:\\d{2}) '
            r'- (?P<level>\\w+) - (?P<message>.*)'
        )
        self.ip_pattern = re.compile(r'\\b(?:[0-9]{1,3}\\.){3}[0-9]{1,3}\\b')
    
    def create_sample_log(self, filename="sample.log"):
        """Create a sample log file for testing"""
        log_entries = [
            "2023-08-27 10:15:30 - INFO - Application started",
            "2023-08-27 10:15:35 - INFO - User login: admin from 192.168.1.100",
            "2023-08-27 10:16:20 - WARNING - Failed login attempt from 192.168.1.200",
            "2023-08-27 10:17:45 - ERROR - Database connection failed",
            "2023-08-27 10:18:00 - INFO - Database connection restored",
            "2023-08-27 10:20:15 - INFO - User logout: admin",
            "2023-08-27 10:25:30 - WARNING - High memory usage detected",
            "2023-08-27 10:30:00 - ERROR - Service timeout for user123",
            "2023-08-27 10:35:45 - INFO - Scheduled backup completed",
        ]
        
        with open(filename, 'w') as f:
            f.write('\\n'.join(log_entries))
        
        return filename
    
    def parse_log_file(self, filename):
        """Parse log file and extract structured data"""
        entries = []
        
        try:
            with open(filename, 'r') as f:
                for line_num, line in enumerate(f, 1):
                    line = line.strip()
                    match = self.log_pattern.match(line)
                    if match:
                        entry = match.groupdict()
                        entry['line_number'] = line_num
                        entry['ip_addresses'] = self.ip_pattern.findall(entry['message'])
                        entries.append(entry)
                    else:
                        # Handle unparseable lines
                        entries.append({
                            'line_number': line_num,
                            'raw_line': line,
                            'level': 'UNKNOWN',
                            'message': line
                        })
        
        except FileNotFoundError:
            print(f"Log file {filename} not found")
            return []
        
        return entries
    
    def analyze_logs(self, entries):
        """Analyze log entries and generate report"""
        if not entries:
            return "No log entries to analyze"
        
        # Count by log level
        level_counts = Counter(entry['level'] for entry in entries)
        
        # Extract IP addresses
        all_ips = []
        for entry in entries:
            if 'ip_addresses' in entry:
                all_ips.extend(entry['ip_addresses'])
        ip_counts = Counter(all_ips)
        
        # Find error patterns
        error_messages = [entry['message'] for entry in entries if entry['level'] == 'ERROR']
        
        # Time analysis (simplified)
        hours = []
        for entry in entries:
            if 'timestamp' in entry:
                time_part = entry['timestamp'].split(' ')[1]
                hour = time_part.split(':')[0]
                hours.append(hour)
        
        hour_activity = Counter(hours)
        
        # Generate report
        report = f"""
=== Log Analysis Report ===

Total Entries: {len(entries)}

Log Levels:
{chr(10).join([f"  {level}: {count}" for level, count in level_counts.most_common()])}

Top IP Addresses:
{chr(10).join([f"  {ip}: {count}" for ip, count in ip_counts.most_common(5)])}

Activity by Hour:
{chr(10).join([f"  {hour}:xx - {count} entries" for hour, count in sorted(hour_activity.items())])}

Error Messages:
{chr(10).join([f"  - {msg}" for msg in error_messages[:5]])}
"""
        return report

# Test log analyzer
analyzer = LogAnalyzer()
log_file = analyzer.create_sample_log()
entries = analyzer.parse_log_file(log_file)
report = analyzer.analyze_logs(entries)
print(report)

# Exercise 3: Simple Testing Framework
class SimpleTest:
    """Basic testing framework"""
    
    def __init__(self):
        self.tests = []
        self.results = {
            'passed': 0,
            'failed': 0,
            'errors': 0
        }
        self.test_details = []
    
    def test(self, description):
        """Decorator for test functions"""
        def decorator(func):
            self.tests.append((description, func))
            return func
        return decorator
    
    def assert_equal(self, actual, expected, message=""):
        """Assert that two values are equal"""
        if actual != expected:
            raise AssertionError(
                f"Expected {expected}, but got {actual}. {message}"
            )
    
    def assert_true(self, condition, message=""):
        """Assert that condition is True"""
        if not condition:
            raise AssertionError(f"Expected True, but got False. {message}")
    
    def assert_raises(self, exception_type, func, *args, **kwargs):
        """Assert that function raises specific exception"""
        try:
            func(*args, **kwargs)
            raise AssertionError(f"Expected {exception_type.__name__} but no exception was raised")
        except exception_type:
            pass  # Expected exception
        except Exception as e:
            raise AssertionError(f"Expected {exception_type.__name__} but got {type(e).__name__}: {e}")
    
    def run_tests(self):
        """Run all registered tests"""
        print(f"Running {len(self.tests)} tests...\\n")
        
        for description, test_func in self.tests:
            try:
                test_func()
                self.results['passed'] += 1
                self.test_details.append(f"✓ PASS: {description}")
                print(f"✓ PASS: {description}")
            except AssertionError as e:
                self.results['failed'] += 1
                self.test_details.append(f"✗ FAIL: {description} - {e}")
                print(f"✗ FAIL: {description} - {e}")
            except Exception as e:
                self.results['errors'] += 1
                self.test_details.append(f"✗ ERROR: {description} - {e}")
                print(f"✗ ERROR: {description} - {e}")
        
        self.print_summary()
    
    def print_summary(self):
        """Print test results summary"""
        total = sum(self.results.values())
        print(f"""
=== Test Summary ===
Total: {total}
Passed: {self.results['passed']}
Failed: {self.results['failed']}
Errors: {self.results['errors']}

Success Rate: {(self.results['passed'] / total * 100):.1f}%
""")

# Example usage of testing framework
test_suite = SimpleTest()

# Test Calculator class from earlier
calc = Calculator()

@test_suite.test("Calculator addition works correctly")
def test_calculator_add():
    test_suite.assert_equal(calc.add(2, 3), 5)
    test_suite.assert_equal(calc.add(-1, 1), 0)

@test_suite.test("Calculator division works correctly")
def test_calculator_divide():
    test_suite.assert_equal(calc.divide(10, 2), 5)

@test_suite.test("Calculator division by zero raises error")
def test_calculator_divide_by_zero():
    test_suite.assert_raises(ValueError, calc.divide, 10, 0)

@test_suite.test("Calculator factorial works correctly")
def test_calculator_factorial():
    test_suite.assert_equal(calc.factorial(5), 120)
    test_suite.assert_equal(calc.factorial(0), 1)

# Run the tests
test_suite.run_tests()